## **Installing Ultralytics for Using Pre-trained Yolo Models**

In [ ]:
# Install the ultralytics package from PyPI
!pip install ultralytics
!pip install roboflow

# **Importing Used Libraries and Seed for Reproducibility**

In [1]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import random
import torch ## PyTorch
from itertools import islice
from ultralytics import YOLO
import pandas as pd
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import shutil
from PIL import Image


## For selective classification
import warnings
# Suppress the specific sklearn warning
warnings.filterwarnings("ignore", message= "y_pred contains classes not in y_true")
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

c:\Projetos_VSC\Visao_Computacional\Computer_Vision\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import numpy as np
import random

# Setting the random seed for reproducibility
seed = 0
np.random.seed(seed)  # for numpy
torch.manual_seed(seed)  # for PyTorch
torch.cuda.manual_seed(seed)
random.seed(0)  # for random module in python

# Check if a GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("GPU is not available.")

# Setting the device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device.")

# Example model and data
# Define a simple model (for illustration)
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = torch.nn.Linear(10, 2)  # Linear layer

    def forward(self, x):
        return self.fc(x)

# Instantiate the model and move it to the selected device (GPU or CPU)
model = SimpleModel().to(device)

# Create some random data (tensor) and move it to the selected device
input_data = torch.randn(5, 10).to(device)  # Batch of 5 samples with 10 features

# Forward pass
output = model(input_data)
print(output)

GPU is available.
Number of GPUs available: 1
GPU 0: NVIDIA GeForce RTX 3050 Laptop GPU
Using cuda device.
tensor([[ 0.3126, -0.0134],
        [-0.2679, -0.2948],
        [ 0.4521,  0.9847],
        [ 0.5986,  1.0274],
        [ 0.4783,  0.3616]], device='cuda:0', grad_fn=<AddmmBackward0>)


# **1 - Loading and Preparing the Data (EDA)**

In [3]:
## uses the kitti-dataset-yolo-format file to convert to acceptable yolo inputs
base_dir = Path('archive')
img_path = base_dir / 'data_object_image_2' / 'training' / 'image_2'
label_path = base_dir / 'labels'
with open(base_dir / 'classes.json', 'r') as f:
    classes = json.load(f)

classes

{'Car': 0,
 'Pedestrian': 1,
 'Van': 2,
 'Cyclist': 3,
 'Truck': 4,
 'Misc': 5,
 'Tram': 6,
 'Person_sitting': 7}

In [4]:
## create pais with images and labels files
images = sorted(list(img_path.glob('*')))
labels = sorted(list(label_path.glob('*')))
pairs = list(zip(images,labels))
pairs[:2]

[(WindowsPath('archive/data_object_image_2/training/image_2/000000.png'),
  WindowsPath('archive/labels/000000.txt')),
 (WindowsPath('archive/data_object_image_2/training/image_2/000001.png'),
  WindowsPath('archive/labels/000001.txt'))]

In [5]:
## spliting between train/val/test sets

train, test_val = train_test_split(pairs,test_size=0.2,shuffle=True)

val, test = train_test_split(test_val, test_size=0.1, shuffle = False)

len(train), len(val), len(test)

(5984, 1347, 150)

In [6]:
## creating paths for train/val/test sets (organizing files)

# destination paths
output_dir = Path('/kaggle/working')
output_train = output_dir / 'train'
output_val = output_dir / 'val'
output_test = output_dir / 'test'

In [7]:
## creating the directories (train/val/test)
output_train.mkdir(parents=True, exist_ok=True)
output_val.mkdir(parents=True, exist_ok=True)
output_test.mkdir(parents=True, exist_ok=True)

In [8]:
## creating function to copy files
def copy_files(pairs, output_folder):
    for img_file, label_file in pairs:
        shutil.copy(img_file, output_folder / img_file.name)
        shutil.copy(label_file, output_folder / label_file.name)

In [9]:
## copying to their respective directories
copy_files(train, output_train)
copy_files(val, output_val)
copy_files(test, output_test)

In [10]:
print(f'Training dataset: {len(train)} samples')
print(f'Validation dataset: {len(val)} samples')
print(f'Test dataset: {len(test)} samples')

Training dataset: 5984 samples
Validation dataset: 1347 samples
Test dataset: 150 samples


In [11]:
## creating YAML file for to point to the data
yaml_file = 'names:\n'
yaml_file += '\n'.join(f'- {c}' for c in classes)
yaml_file += f'\nnc: {len(classes)}'
yaml_file += f'\ntrain: {str(output_train)}\nval: {str(output_val)} \ntest: {str(output_test)}'
with open('kitti.yaml','w') as f:
    f.write(yaml_file)

In [12]:
!cat kitti.yaml

'cat' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


# **2 - Choosing Model, Training/Validation Steps**

## **Instantiating the Model**

In [13]:
# Load a model
model = YOLO("yolo11s.yaml")  # build a new model from YAML
#model = YOLO("yolo11s.pt")  # load a pretrained model (recommended for training)
#model = YOLO("yolo11s.yaml").load("yolo11s.pt")  # build from YAML and transfer weights
model = YOLO("yolo11s_best.pt")  # load a custom model

## **Training**

In [14]:
train_results = model.train(
    data='kitti.yaml', 
    epochs=20,
    patience=3,
    mixup=0.1,
    project='yolo11s-kitti',
    device=0
)

Ultralytics 8.3.29  Python-3.12.7 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolo11s_best.pt, data=kitti.yaml, epochs=20, time=None, patience=3, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=yolo11s-kitti, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

train: Scanning C:\kaggle\working\train... 5984 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5984/5984 [00:44<00:00, 134.44it/s]


train: New cache created: C:\kaggle\working\train.cache


val: Scanning C:\kaggle\working\val.cache... 1347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1347/1347 [00:00<?, ?it/s]


Plotting labels to yolo11s-kitti\train6\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo11s-kitti\train6
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20      4.39G     0.8803     0.5754     0.9387        201        640:  62%|██████▏   | 233/374 [03:33<02:09,  1.09it/s]


KeyboardInterrupt: 

## **Validating**

In [ ]:
val_results = model.val(data="kitti.yaml")

## **Seeing Results from Training**

In [ ]:
plt.figure(figsize=(10,30))
plt.imshow(Image.open('/kaggle/working/yolo11s-kitti/train/results.png'))
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open('/kaggle/working/yolo11s-kitti/train/confusion_matrix_normalized.png'))
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open('/kaggle/working/runs/detect/val3/confusion_matrix_normalized.png'))
plt.axis('off')
plt.show()

## **Seeing Predictions on Validation Set**

In [ ]:
preds = model.predict([val[idx][0] for idx in np.random.randint(0,len(val),(20,))],save=True)

In [ ]:
preds = list(Path('/kaggle/working/runs/detect/predict2').glob('*'))
preds

In [ ]:
def plot_images(images):
    num_images = len(images)
    rows = num_images
    cols = 1
    fig, axes = plt.subplots(rows, cols, figsize=(15, 80))
    for ax in axes.flat:
        ax.axis('off')
    for i, img_path in enumerate(images):
        img = Image.open(img_path)
        axes[i].imshow(img)
    
    plt.tight_layout()
    plt.show()
    
plot_images(preds)